In [1]:
import numpy as np
import pandas as pd
teste = pd.read_csv("../input/titanic/test.csv")
treino = pd.read_csv("../input/titanic/train.csv")
bum = treino
#7
#retirando cabin e ticket dos dataframes
teste.drop(['Cabin', 'Ticket'], axis=1)
treino.drop(['Cabin', 'Ticket'], axis=1)
#substituindo os vazios pela média de Age nos dataframes
Age_mean = teste['Age'].median()
teste['Age'] = teste['Age'].fillna(Age_mean)
newAge_mean = np.median(treino['Age'])
treino['Age'] = treino['Age'].replace(np.nan, newAge_mean)
#substituindo os vazios pela média de Fare nos dataframes
fare_mean = teste['Fare'].median()
teste['Fare'] = teste['Fare'].fillna(fare_mean)
fare_media = np.median(treino['Fare'])
treino['Fare'] = treino['Fare'].replace(np.nan, fare_media)
#substituindo os vazios pela moda de Embarked
Embarked_moda = teste['Embarked'].mode()
teste['Embarked'] = teste['Embarked'].fillna(Embarked_moda)
Embarked_mode = treino['Embarked'].mode()
treino['Embarked'] = treino['Embarked'].fillna(Embarked_mode)
#Criar Feature que separe a Feature “Age” em 5 intervalos de mesma extensão.
teste['Age-1'] = pd.cut(teste['Age'], 5)
#Criar Feature que separe a Feature “Age” em 5 intervalos de mesma extensão.
treino['Age-1'] = pd.cut(treino['Age'],5)
#Criar Feature que separe “Fare” em 6 intervalos que contenham o mesmo número de dados
teste['Fare-1'] = pd.qcut(teste['Fare'],6)
#Criar Feature que separe “Fare” em 6 intervalos que contenham o mesmo número de dados
treino['Fare-1'] = pd.qcut(treino['Fare'],6)
testeA= teste.loc[:,['Age', 'Fare', 'Sex', 'Embarked', 'Pclass', 'Age-1', 'SibSp', 'Parch', 'Fare-1']]
treinoA = treino.loc[:,['Age', 'Fare', 'Sex', 'Embarked', 'Pclass', 'Survived', 'Age-1', 'SibSp', 'Parch', 'Fare-1']]
treinob=treinoA.dropna()
testeb=testeA.dropna()


In [2]:
#8
from sklearn.preprocessing import OneHotEncoder
s = (treinob.dtypes == 'object')
d = list(s[s].index)
print(d)
#Auto encodes any dataframe column of type category or object.
def dummyEncode(df):
        columnsToEncode = list(df.select_dtypes(include=['object']))
        treinoc = df
        testec = testeb
        s = (treinoc.dtypes == 'object')
        d = list(s[s].index)
        le = OneHotEncoder(handle_unknown='ignore', sparse=False)
        for feature in d:
            try:
                one = pd.DataFrame(le.fit_transform(treinoc[d]))
                one.index=df.index
            except:
                print('Error encoding '+feature)
            one.columns = le.get_feature_names(d)
            v = pd.DataFrame(le.transform(testec[d]))
            c = pd.concat([df,one], axis=1)
            return c
treinoU=dummyEncode(treinob)
testeU=dummyEncode(testeb)
display(testeU)

['Sex', 'Embarked']


/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Age,Fare,Sex,Embarked,Pclass,Age-1,SibSp,Parch,Fare-1,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,34.5,7.8292,male,Q,3,"(30.502, 45.668]",0,0,"(7.762, 8.662]",0.0,1.0,0.0,1.0,0.0
1,47.0,7.0000,female,S,3,"(45.668, 60.834]",1,0,"(-0.001, 7.762]",1.0,0.0,0.0,0.0,1.0
2,62.0,9.6875,male,Q,2,"(60.834, 76.0]",0,0,"(8.662, 14.454]",0.0,1.0,0.0,1.0,0.0
3,27.0,8.6625,male,S,3,"(15.336, 30.502]",0,0,"(7.762, 8.662]",0.0,1.0,0.0,0.0,1.0
4,22.0,12.2875,female,S,3,"(15.336, 30.502]",1,1,"(8.662, 14.454]",1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,27.0,8.0500,male,S,3,"(15.336, 30.502]",0,0,"(7.762, 8.662]",0.0,1.0,0.0,0.0,1.0
414,39.0,108.9000,female,C,1,"(30.502, 45.668]",0,0,"(55.969, 512.329]",1.0,0.0,1.0,0.0,0.0
415,38.5,7.2500,male,S,3,"(30.502, 45.668]",0,0,"(-0.001, 7.762]",0.0,1.0,0.0,0.0,1.0
416,27.0,8.0500,male,S,3,"(15.336, 30.502]",0,0,"(7.762, 8.662]",0.0,1.0,0.0,0.0,1.0


In [3]:
 # dividindo o dataframe de treino em train e validation
from sklearn.model_selection import train_test_split

X = treinoU.drop(['Survived','Age-1', 'Fare-1', 'Embarked', 'Sex', ],axis = 1)
display(X)
target = treinoU["Survived"]
train_X, val_X, train_y, val_y = train_test_split(X, target,test_size=0.20, random_state = 0)
# treinando o modelo
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

modelo_titanic = RandomForestClassifier()
modelo_titanic.fit(train_X,train_y)
predt = modelo_titanic.predict(val_X)
accuracy_score(predt,val_y)

,Age,Fare,Pclass,SibSp,Parch,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,22.0,7.2500,3,1,0,0.0,1.0,0.0,0.0,1.0
1,38.0,71.2833,1,1,0,1.0,0.0,1.0,0.0,0.0
2,26.0,7.9250,3,0,0,1.0,0.0,0.0,0.0,1.0
3,35.0,53.1000,1,1,0,1.0,0.0,0.0,0.0,1.0
4,35.0,8.0500,3,0,0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
885,39.0,29.1250,3,0,5,1.0,0.0,0.0,1.0,0.0
886,27.0,13.0000,2,0,0,0.0,1.0,0.0,0.0,1.0
887,19.0,30.0000,1,0,0,1.0,0.0,0.0,0.0,1.0
889,26.0,30.0000,1,0,0,0.0,1.0,1.0,0.0,0.0


0.7972027972027972

In [4]:
F= testeU.drop(['Age-1', 'Fare-1', 'Embarked', 'Sex'], axis=1)
gua = teste['PassengerId']
pred= modelo_titanic.predict(F)
output = pd.DataFrame({'PassengerId' : gua, 'Survived' : pred})
output.to_csv('submission.csv', index=False)
output.shape
display(output)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [5]:
#8.2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
#Auto encodes any dataframe column of type category or object.
def pp(data, encoder, model, numerical_imputer, categorical_imputer):   
        cat = [coln for coln in data.columns if data[coln].nunique() < 10 and data[coln].dtype == 'object']
        num = [coln for coln in data.columns if data[coln].dtype in ['float64', 'int64'] and coln not in ['Survived', 'Name', 'PassengerId', 'Ticket', 'Cabin']]
        feat = cat + num
        target = data['Survived']
        X = data[feat]
        X_train, X_valid, y_train, y_valid = train_test_split(X, target, test_size=0.2, random_state=0)
        # Preprocessing for numerical data
        numerical_transformer = SimpleImputer(strategy=numerical_imputer)

        # Preprocessing for categorical data
        categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),('encode', encoder)])

        # Bundle preprocessing for numerical and categorical data
        preprocessor = ColumnTransformer(transformers=[('num', numerical_transformer, num),('cat', categorical_transformer, cat)])
        # Bundle preprocessing and modeling code in a pipeline
        my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),('model', model)])

        # Preprocessing of training data, fit model 
        my_pipeline.fit(X_train, y_train)

        # Preprocessing of validation data, get predictions
        preds = my_pipeline.predict(X_valid)

        # Evaluate the model
        score = accuracy_score(y_valid, preds)
        scorep = (score * 100)
        rt = print('enco:',encoder, numerical_transformer, scorep)
        print(score.mean())
        return rt
st = ['mean', 'median', 'most_frequent', 'constant']
encoders = [OneHotEncoder(handle_unknown='ignore',sparse=False), OrdinalEncoder()]
cateim='most_frequent'
for i in encoders:
    for j in st:
        score = pp(bum,i,RandomForestClassifier(),j,cateim)

enco: OneHotEncoder(handle_unknown='ignore', sparse=False) SimpleImputer() 83.79888268156425
0.8379888268156425
enco: OneHotEncoder(handle_unknown='ignore', sparse=False) SimpleImputer(strategy='median') 82.12290502793296
0.8212290502793296
enco: OneHotEncoder(handle_unknown='ignore', sparse=False) SimpleImputer(strategy='most_frequent') 85.47486033519553
0.8547486033519553
enco: OneHotEncoder(handle_unknown='ignore', sparse=False) SimpleImputer(strategy='constant') 84.35754189944134
0.8435754189944135
enco: OrdinalEncoder() SimpleImputer() 84.91620111731844
0.8491620111731844
enco: OrdinalEncoder() SimpleImputer(strategy='median') 81.56424581005587
0.8156424581005587
enco: OrdinalEncoder() SimpleImputer(strategy='most_frequent') 84.35754189944134
0.8435754189944135
enco: OrdinalEncoder() SimpleImputer(strategy='constant') 83.79888268156425
0.8379888268156425


In [6]:
#metodo diferente do pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
x, y = make_classification(random_state=0)
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    random_state=0)
pipeline = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])

pipeline.fit(x_train, y_train)
pipeline.score(x_test, y_test)

0.88

In [7]:
#8.3
from sklearn.ensemble import GradientBoostingClassifier
ppp= GradientBoostingClassifier(random_state = 0, n_iter_no_change = 100).fit(train_X, train_y)
ppp.fit(train_X, train_y)
uu=ppp.predict(val_X)
accuracy_score(uu,val_y)
#esse é maior


0.8391608391608392